In [110]:
import requests
import pendulum
import pandas as pd
from bs4 import BeautifulSoup
import regex as re

In [47]:
def extract_current_date():
    '''extract current date to scrap news'''
    # get raw date
    date_raw = pendulum.today()

    # extract year, month, day
    year = str(date_raw.year)
    month = str(date_raw.month)
    if len(month) == 1:
        month = '0' + month

    day = str(date_raw.day)    
    if len(day) == 1:
        day = '0' + day

    return year + '-' + month + '-' + day

def get_current_news_update():
    '''Get the news links'''
    # Get the current date
    current_date = extract_current_date()

    # Get current news update 
    news_raw = requests.get(f'https://www.tempo.co/indeks/{current_date}/')

    # Convert to soup for html parsing
    news_soup_raw = BeautifulSoup(news_raw.text)
    news_tag_all = news_soup_raw.findAll('div', {'class': 'card-box ft240 margin-bottom-sm'})

    # Find all link
    news_updates = []
    for news_tag in news_tag_all:
        # Create dict
        news_dict = {}

        # Parse the link
        for link_raw in news_tag.find('h2', 'title'):
            # Get the link
            news_dict['link'] = link_raw.get('href')
        
        # Parse the title
        for title_raw in news_tag.find('h2'):
            news_dict['title'] = str(title_raw).strip('</a>').split('>')[-1].rstrip()

        # Parse the headline descriptions
        for headline_raw in news_tag.find('p'):
            # Get the headline
            news_dict['headline'] = str(headline_raw).rstrip()

        # Append to news_links
        news_updates.append(news_dict)

    return news_updates


In [97]:
current_news_links = get_current_news_update()

In [145]:
news_raw = requests.get(current_news_links[1]['link'])
news_soup_raw = BeautifulSoup(news_raw.text)


In [147]:
KEYWORDS_TO_REMOVE = ['Baca Juga', 'Pilihan Editor']

def clean_detail_konten(konten_raws):
    '''parse detail konten html to str'''
    # Create a content_list
    content_list = []
    for konten_raw in konten_raws.findAll('p'):
        konten_raw_text = konten_raw.text
        konten_clean = konten_raw_text.rstrip().lstrip()
        content_list.append(konten_clean)

    # First, find index
    index_to_remove = []
    for i, konten in enumerate(content_list):
        for keywords in KEYWORDS_TO_REMOVE:
            if keywords in konten:
                index_to_remove.append(i)

                if keywords == 'Baca Juga':
                    index_to_remove.append(i+1)
                continue

    # Then remove the index
    clean_content_list = [konten for i, konten in enumerate(content_list) if i not in index_to_remove]
    
    # Finally join
    clean_konten = ' '.join(clean_content_list)

    return clean_konten


In [146]:
# Find title
title = news_soup_raw.find('h1').text
print(title)

# Find reporter
reporter = news_soup_raw.find('span', {'itemprop': 'author'}).text
print(reporter)

# Find editor
editor = news_soup_raw.find('span', {'itemprop': 'editor'}).text
print(editor)

# Find published date
date_published = news_soup_raw.find('p', {'class': 'date margin-bottom-sm'}).text
print(date_published)

# Find detail konten
konten_raws = news_soup_raw.find('div', {'class': 'detail-konten'})
konten_clean = clean_detail_konten(konten_raws)
print(konten_clean)

# Find news_tag
tag_raws = news_soup_raw.find('div', {'class': 'box-tag-detail'})
tag_cleans = ', '.join([tag.text for tag in tag_raws.findAll('a')])
print(tag_cleans)

Taiwan Deteksi 28 Pesawat Tempur Cina di Zona Pertahanan Udara 
Tempo.co
Ida Rosdalina
Minggu, 17 September 2023 22:04 WIB
TEMPO.CO, Jakarta - Cina menerbangkan 28 pesawat tempur di sekitar Taiwan pada Minggu, 17 September 2023, dan sebagian besar melintasi garis tengah Selat Taiwan untuk misi “jarak jauh” yang tidak ditentukan, kata pihak berwenang Taipei, seperti dikutip AFP. Kementerian Pertahanan Taiwan mengatakan 20 pesawat yang terdeteksi sejak Minggu pagi melintasi garis median jalur air yang memisahkan Taiwan dan daratan Cina, dan memasuki zona identifikasi pertahanan udara tenggara dan barat daya pulau yang memiliki pemerintahan sendiri itu. Cina sedang melakukan “misi seperti latihan dan pelatihan jarak jauh”, kata kementerian pertahanan dalam sebuah pernyataan, seraya menambahkan bahwa pihaknya memantau situasi dengan pesawat dan kapal patroli. Beijing mengklaim Taiwan yang demokratis sebagai wilayahnya sendiri akan direbut suatu hari nanti, dengan kekerasan jika perlu. Mere

In [152]:
def extract_news_details(news_link):
    '''extract the news details'''
    # Request news
    news_raw = requests.get(news_link)
    news_soup_raw = BeautifulSoup(news_raw.text)

    # Extract info
    news_data = {}

    news_data['title'] = news_soup_raw.find('h1').text
    try:
        news_data['reporter'] = news_soup_raw.find('span', {'itemprop': 'author'}).text
    except:
        news_data['reporter'] = None 

    try:
        news_data['editor'] = news_soup_raw.find('span', {'itemprop': 'editor'}).text
    except:
        news_data['editor'] = None
        
    news_data['date_published'] = news_soup_raw.find('p', {'class': 'date margin-bottom-sm'}).text
    konten_raws = news_soup_raw.find('div', {'class': 'detail-konten'})
    news_data['konten'] = clean_detail_konten(konten_raws)
    tag_raws = news_soup_raw.find('div', {'class': 'box-tag-detail'})
    news_data['tag'] = ', '.join([tag.text for tag in tag_raws.findAll('a')])

    return news_data


In [154]:
news_data = extract_news_details('https://bola.tempo.co/read/1773075/hasil-liga-1-persik-kediri-vs-persija-jakarta-1-2-gol-penalti-ondrej-kudela-bawa-macan-kemayoran-menang')
news_data

{'title': 'Hasil Liga 1: Persik Kediri vs Persija Jakarta 1-2, Gol Penalti Ondrej Kudela Bawa Macan Kemayoran Menang',
 'reporter': None,
 'editor': 'Rina Widiastuti',
 'date_published': 'Minggu, 17 September 2023 21:12 WIB',
 'konten': "TEMPO.CO, Jakarta -\xa0Duel Persik Kediri vs Persija Jakarta pada pekan ke-12 Liga 1 yang berlangsung di Stadion Brawijaya, Minggu, 17 September 2023, berakhir dengan skor 1-2 untuk kemenangan Macan Kemayoran. Dalam laga ini, tim tuan rumah berhasil unggul lebih dulu pada menit ke-35. Muhammad Khanafi yang mendapat umpan terobosan dari Renan da Silva, berhasil menyelesaikan dengan baik. Dia mampu melewati hadangan kiper Andritany Adhiyasa yang maju ke depan untuk menghentikan pergerakannya. Persija Jakarta memiliki peluang emas untuk menyamakan kdudukan pada menit ke-41. Saat itu, Ryo Matsumura, yang berada di depan gawang Persik, mendapatkan kiriman umpan terobosan dari rekannya, tetapi kontrol bolanya tidak sempurna sehingga bola mengelinding keluar 

In [2]:
import pandas as pd

In [3]:
news_brief = pd.read_csv('news_brief.csv')
news_brief

,link,title,headline
0,https://nasional.tempo.co/read/1773083/lahan-r...,"Lahan Rempang Harus Kosong 28 September, Bahli...",Menteri Investasi/BKPM Bahlil Lahadalia mengat...
1,https://metro.tempo.co/read/1773080/pengelola-...,Pengelola Museum Nasional Belum Hitung Kerugia...,Kebakaran ini berdampak pada enam ruangan di G...
2,https://nasional.tempo.co/read/1773079/waketum...,Waketum Gerindra Bicara soal Kemungkinan SBY M...,Waketum Gerindra Budi Djiwandono menanggapi ih...
3,https://dunia.tempo.co/read/1773076/para-perem...,Para Perempuan Terpidana Mati Iran Bakar Penjara,Perempuan terpidana mati Iran ini memprotes ma...
4,https://bola.tempo.co/read/1773075/hasil-liga-...,Hasil Liga 1: Persik Kediri vs Persija Jakarta...,Gol penalti Ondrej Kudela pada menit ke-90+6' ...
...,...,...,...
1375,https://bola.tempo.co/read/1774478/hasil-mls-i...,Hasil MLS: Inter Miami Kalahkan Toronto FC 4-0...,Lionel Messi kembali menjadi starter setelah a...
1376,https://metro.tempo.co/read/1774475/keluarga-i...,Keluarga Imam Masykur Berharap Pasal yang Menj...,Keluarga Imam Masykur berharap pelaku penculik...
1377,https://dunia.tempo.co/read/1774461/ribut-soal...,"Ribut Soal Impor Gandum, Polandia Hentikan Ban...",Polandia tidak akan lagi mempersenjatai Ukrain...
1378,https://bisnis.tempo.co/read/1774473/sri-mulya...,Sri Mulyani soal Inflasi di RI: Relatif Rendah...,Sri Mulyani menyebutkan inflasi Indonesia per ...


In [4]:
news_details = pd.read_csv('news_detail.csv')
news_details

,link,title,reporter,editor,date_published,konten,tag
0,https://metro.tempo.co/read/1773102/tak-ingin-...,"Tak Ingin RTH Kalijodo Disalahgunakan, Heru Bu...",Mutia Yuantisya,Clara Maria Tjandra Dewi H.,"Minggu, 17 September 2023 23:02 WIB","TEMPO.CO, Jakarta - Penjabat (Pj) Gubernur DKI...","RTH, Ruang Terbuka Hijau, Kalijodo, Heru Budi,..."
1,https://bisnis.tempo.co/read/1773098/bahlil-se...,Bahlil Sebut Proyek Rempang Eco City untuk Kes...,Riri Rahayu,Martha Warta Silaban,"Minggu, 17 September 2023 22:50 WIB","TEMPO.CO, Jakarta - Menteri Investasi Bahlil L...","Bahlil, Pulau Rempang, Investasi, Bahlil Lahad..."
2,https://bisnis.tempo.co/read/1773096/resmikan-...,"Resmikan Toko NU-Mandiri di Yogya, Mendag Zulh...",Pribadi Wicaksono (Kontributor),Martha Warta Silaban,"Minggu, 17 September 2023 22:41 WIB","TEMPO.CO, Yogyakarta - Menteri Perdagangan Zul...","Zulhas, NU, Yogyakarta, UMKM, toko"
3,https://dunia.tempo.co/read/1773095/penasihat-...,Penasihat Keamanan Nasional AS Bertemu Menlu C...,Tempo.co,Ida Rosdalina,"Minggu, 17 September 2023 22:40 WIB","TEMPO.CO, Jakarta - Penasihat keamanan nasiona...","Wang Yi, Menteri Luar Negeri Cina, Xi Jinping,..."
4,https://seleb.tempo.co/read/1773077/suga-bts-m...,Suga BTS Masuk Wajib Militer Lima Hari Lagi!,Tempo.co,Istiqomatul Hayati,"Minggu, 17 September 2023 22:30 WIB","TEMPO.CO, Jakarta - Salah satu personel BTS, M...","Suga, Suga BTS, BigHit Music, BTS, Album D-Day"
...,...,...,...,...,...,...,...
1375,https://bola.tempo.co/read/1774478/hasil-mls-i...,Hasil MLS: Inter Miami Kalahkan Toronto FC 4-0...,Randy Fauzi Febriansyah,Arkhelaus Wisnu Triyogo,"Kamis, 21 September 2023 08:44 WIB","TEMPO.CO, Jakarta - Lionel Messi kembali menja...","Lionel Messi, Inter Miami, Inter Miami vs Toro..."
1376,https://metro.tempo.co/read/1774475/keluarga-i...,Keluarga Imam Masykur Berharap Pasal yang Menj...,Desty Luthfiani,Ahmad Faiz Ibnu Sani,"Kamis, 21 September 2023 08:41 WIB","TEMPO.CO, Jakarta - Keluarga Imam Masykur dan ...","Imam Masykur, Riswandi Manik, Penculikan, Pemb..."
1377,https://dunia.tempo.co/read/1774461/ribut-soal...,"Ribut Soal Impor Gandum, Polandia Hentikan Ban...",Tempo.co,Sita Planasari,"Kamis, 21 September 2023 08:30 WIB","TEMPO.CO, Jakarta - Polandia tidak akan lagi m...","Polandia, Ukraina, Bantuan senjata untuk Ukrai..."
1378,https://bisnis.tempo.co/read/1774473/sri-mulya...,Sri Mulyani soal Inflasi di RI: Relatif Rendah...,Moh. Khory Alfarizi,Rr. Ariyani Yakti Widyastuti,"Kamis, 21 September 2023 08:29 WIB","TEMPO.CO, Jakarta - Menteri Keuangan Sri Mulya...","Sri Mulyani, Inflasi, Harga pangan, Suku Bunga..."


In [11]:
import numpy as np

In [18]:
word, count = np.unique(' '.join(news_details['konten'].tolist()).lower().split(' '), return_counts=True)

In [19]:
sorted({w:c for w, c in zip(word, count)}.items(), key=lambda x:x[1], reverse=True)

[('yang', 4378),
 ('dan', 3892),
 ('di', 3670),
 ('dengan', 1778),
 ('untuk', 1737),
 ('dari', 1699),
 ('pada', 1493),
 ('dalam', 1412),
 ('ini', 1397),
 ('akan', 989),
 ('tidak', 886),
 ('september', 841),
 ('itu', 840),
 ('jakarta', 820),
 ('juga', 802),
 ('atau', 748),
 ('kata', 708),
 ('ke', 708),
 ('2023.', 664),
 ('ada', 616),
 ('bisa', 611),
 ('adalah', 578),
 ('sebagai', 566),
 ('menjadi', 565),
 ('tempo.co,', 529),
 ('saat', 514),
 ('indonesia', 502),
 ('karena', 479),
 ('sudah', 479),
 ('oleh', 477),
 ('-', 462),
 ('partai', 459),
 ('tersebut', 459),
 ('lebih', 453),
 ('tahun', 423),
 ('telah', 418),
 ('memiliki', 415),
 ('bahwa', 387),
 ('dia', 387),
 ('mereka', 379),
 ('dapat', 373),
 ('satu', 354),
 ('seperti', 349),
 ('mengatakan', 342),
 ('setelah', 337),
 ('hingga', 332),
 ('secara', 328),
 ('ia', 326),
 ('kami', 314),
 ('2023', 311),
 ('itu,', 304),
 ('kepada', 291),
 ('beberapa', 280),
 ('dua', 279),
 ('harus', 275),
 ('melakukan', 274),
 ('kota', 268),
 ('banyak', 26

In [158]:
news_details['editor'].value_counts()

Nurdin Saleh                    16
Eko Ari Wibowo                  15
Wawan Priyanto                  13
Martha Warta Silaban            13
Rina Widiastuti                 13
Linda novi trianita             12
Mila Novita                     11
Ahmad Faiz Ibnu Sani            11
Bram Setiawan                   11
Grace gandhi                    10
Yudono Yanuar                   10
Lani Diana Wijaya                9
Laili Ira                        8
Istiqomatul Hayati               8
Nurhadi                          7
Andry Triyanto Tjitra            7
Ida Rosdalina                    7
Yayuk Widiyarti                  6
Erwin Prima                      5
Suci Sekarwati                   5
S. Dian Andryanto                5
Ninis Chairunnisa                5
Clara Maria Tjandra Dewi H.      5
Naufal Ridhwan                   5
Rr. Ariyani Yakti Widyastuti     4
Dwi Arjanto                      3
Zacharias Wuragil                3
Dewi Rina Cahyani                3
Hotma Radja Siregar 